In [1]:
import itertools
import numpy as np

somelists = [
#    [1, 2],
#    [2, 3],
#    [1, 3]
    ['0', '0~'],
    ['1', '1~'],
    ['0', '0~']
]

lists = [
         [1, 2],
         [1, 3],
         [2, 3]
]

print(*itertools.product(*somelists))
print(*itertools.product(*lists))

#for element in itertools.product(*somelists):
#    print(element)

('0', '1', '0') ('0', '1', '0~') ('0', '1~', '0') ('0', '1~', '0~') ('0~', '1', '0') ('0~', '1', '0~') ('0~', '1~', '0') ('0~', '1~', '0~')
(1, 1, 2) (1, 1, 3) (1, 3, 2) (1, 3, 3) (2, 1, 2) (2, 1, 3) (2, 3, 2) (2, 3, 3)


In [2]:
from sympy.logic.boolalg import to_cnf
from sympy.abc import A, B, C, D, E, F, G, H, J


# to_cnf((A & B) | (C & D) | (E & F) | (G & H))
to_cnf((A & B) | (C & D) | (E & F))
# to_cnf((~A & ~B) | (~C & ~D) | (~E & ~F))
# to_cnf((A & B & C) | (D & E & F))

(A | C | E) & (A | C | F) & (A | D | E) & (A | D | F) & (B | C | E) & (B | C | F) & (B | D | E) & (B | D | F)

In [3]:
to_cnf(((~A & B) | (A & ~B)) | ((~C & D) | (C & ~D))) #  | ((~E & F) | (E & ~F)))

(A | B | C | D) & (A | B | C | ~C) & (A | B | D | ~D) & (A | C | D | ~A) & (B | C | D | ~B) & (A | B | ~C | ~D) & (A | C | ~A | ~C) & (A | D | ~A | ~D) & (B | C | ~B | ~C) & (B | D | ~B | ~D) & (C | D | ~A | ~B) & (A | ~A | ~C | ~D) & (B | ~B | ~C | ~D) & (C | ~A | ~B | ~C) & (D | ~A | ~B | ~D) & (~A | ~B | ~C | ~D)

# Packages

In [4]:
%%capture
!pip install python-sat

In [7]:
from pysat.examples.fm import FM
from pysat.formula import WCNF
from itertools import combinations
import math

import numpy as np

# Prepare indices

In [12]:
n_neurons = 3
n_features = n_neurons
n_train = 4

In [10]:
# define propositional variables for weights
w = np.array([i + 1 for i in range(n_neurons)])
print(f'Propositional weights: {w}')

[1 2 3]


In [21]:
# define propositional variables for X
# propositional features cannot have same indices as w --> + 1 + max(w)
propositional_features = np.array(
    [[i*n_features + j + max(w) + 1 for j in range(n_features)] for i in range(n_train)]
)
print(propositional_features)

for i in range(propositional_features.shape[0]):
    for j in range(propositional_features.shape[1]):
        print(f'X[{i}][{j}] will be encoded as {propositional_features[i][j]}')

[[ 4  5  6]
 [ 7  8  9]
 [10 11 12]
 [13 14 15]]
X[0][0] will be encoded as 4
X[0][1] will be encoded as 5
X[0][2] will be encoded as 6
X[1][0] will be encoded as 7
X[1][1] will be encoded as 8
X[1][2] will be encoded as 9
X[2][0] will be encoded as 10
X[2][1] will be encoded as 11
X[2][2] will be encoded as 12
X[3][0] will be encoded as 13
X[3][1] will be encoded as 14
X[3][2] will be encoded as 15


In [24]:
# Now we prepare all indices for the case when y == 1

majority_size = int(np.ceil(n_features / 2))
assert(majority_size == np.floor(n_features / 2) + 1)

features_combinations = list(combinations(w, majority_size))
print(features_combinations)  # [(w1 == x_i1) and (w2 == x_i2)] V [(..1) and (..3)] V [(..2) and (..3)]

[(1, 2), (1, 3), (2, 3)]


In [27]:
# Now prepare all products
cartesian_prod = list(itertools.product(*features_combinations))
print(cartesian_prod[:10])

[(1, 1, 2), (1, 1, 3), (1, 3, 2), (1, 3, 3), (2, 1, 2), (2, 1, 3), (2, 3, 2), (2, 3, 3)]


In [38]:
optimized_products = [np.unique(combo).tolist() for combo in cartesian_prod]
print(optimized_products[:10])

[[1, 2], [1, 3], [1, 2, 3], [1, 3], [1, 2], [1, 2, 3], [2, 3], [2, 3]]


In [44]:
# Each element of optimized_products represents:
# [1, 2] --> (w1 == x_i1) V (w2 == x_i2). But we have not transfered to CNF yet
# To do that: [(~w1 V x_i1) & (w1 V ~x_i1)] V [(~w2 V x_i2) & (w2 V ~x_i2)]
# which is 'encoded' as [[1, -1], [2, -2]]

predfinal_clauses = []
for product in optimized_products:
    bracket = []
    for val in product:
        bracket.append([val, -val])
    predfinal_clauses.append(bracket)
predfinal_clauses

[[[1, -1], [2, -2]],
 [[1, -1], [3, -3]],
 [[1, -1], [2, -2], [3, -3]],
 [[1, -1], [3, -3]],
 [[1, -1], [2, -2]],
 [[1, -1], [2, -2], [3, -3]],
 [[2, -2], [3, -3]],
 [[2, -2], [3, -3]]]

In [49]:
# And now we are creating final clauses
final_clauses = []
for predfinal_clause in predfinal_clauses:
    print(f'predfinal_clause = {predfinal_clause}')
    clauses = list(itertools.product(*predfinal_clause))
    print(f'Clauses from it: {clauses}')

    for clause in clauses:
        final_clauses.append(clause)

final_clauses

predfinal_clause = [[1, -1], [2, -2]]
Clauses from it: [(1, 2), (1, -2), (-1, 2), (-1, -2)]
predfinal_clause = [[1, -1], [3, -3]]
Clauses from it: [(1, 3), (1, -3), (-1, 3), (-1, -3)]
predfinal_clause = [[1, -1], [2, -2], [3, -3]]
Clauses from it: [(1, 2, 3), (1, 2, -3), (1, -2, 3), (1, -2, -3), (-1, 2, 3), (-1, 2, -3), (-1, -2, 3), (-1, -2, -3)]
predfinal_clause = [[1, -1], [3, -3]]
Clauses from it: [(1, 3), (1, -3), (-1, 3), (-1, -3)]
predfinal_clause = [[1, -1], [2, -2]]
Clauses from it: [(1, 2), (1, -2), (-1, 2), (-1, -2)]
predfinal_clause = [[1, -1], [2, -2], [3, -3]]
Clauses from it: [(1, 2, 3), (1, 2, -3), (1, -2, 3), (1, -2, -3), (-1, 2, 3), (-1, 2, -3), (-1, -2, 3), (-1, -2, -3)]
predfinal_clause = [[2, -2], [3, -3]]
Clauses from it: [(2, 3), (2, -3), (-2, 3), (-2, -3)]
predfinal_clause = [[2, -2], [3, -3]]
Clauses from it: [(2, 3), (2, -3), (-2, 3), (-2, -3)]


[(1, 2),
 (1, -2),
 (-1, 2),
 (-1, -2),
 (1, 3),
 (1, -3),
 (-1, 3),
 (-1, -3),
 (1, 2, 3),
 (1, 2, -3),
 (1, -2, 3),
 (1, -2, -3),
 (-1, 2, 3),
 (-1, 2, -3),
 (-1, -2, 3),
 (-1, -2, -3),
 (1, 3),
 (1, -3),
 (-1, 3),
 (-1, -3),
 (1, 2),
 (1, -2),
 (-1, 2),
 (-1, -2),
 (1, 2, 3),
 (1, 2, -3),
 (1, -2, 3),
 (1, -2, -3),
 (-1, 2, 3),
 (-1, 2, -3),
 (-1, -2, 3),
 (-1, -2, -3),
 (2, 3),
 (2, -3),
 (-2, 3),
 (-2, -3),
 (2, 3),
 (2, -3),
 (-2, 3),
 (-2, -3)]

In [54]:
w = WCNF()

w.append([1])
w.append([1, 2], [2, 3])

w.hard

TypeError: ignored

# Task

In [ ]:
from pysat.formula import CNF
from pysat.formula import WCNF

In [ ]:
n_neurons = 3
n_train = 3
n_features = n_neurons

assert(n_neurons % 2 == 1)  # odd number of neurons is allowed

def check_if_2nd_feature_true(X):
    return [x_i[1] is True for x_i in X]

def prepare_dataset(n_train_, n_features_, func_):
    X = [
         [True, False, False],
         [False, True, False],
         [True, True, False]
    ]
    Y = func_(X)
    return X, Y

X, Y = prepare_dataset(
    n_train_ = n_train, 
    n_features_ = n_features,
    func_ = check_if_2nd_feature_true
)

In [ ]:
print(X)
print(Y)

[[True, False, False], [False, True, False], [True, True, False]]
[False, True, True]


In [ ]:
# wcnf = WCNF()

# wcnf.append([-1, 2]) 
# wcnf.append([-2, 3])
# wcnf.append([-3, True])
# wcnf.append([4, True])

# print(wcnf.soft)
# print(wcnf.hard)
# print(wcnf.wght)

# fm = FM(wcnf,verbose=10)
# fm.compute()

In [ ]:
# def encode_features(X):
#     n = len(X)
#     m = len(X[0])
#     print(f'Shape of X: n = {n}, m = {m}')
#     prop_features = [[i * n + j + 1 for j in range(m)] for i in range(n)]
#     return np.array(prop_features)


# Encoding weights
prop_weights_indices = [i for i in range(n_neurons)]

In [ ]:
prop_weights_indices

[0, 1, 2]

In [ ]:
group_size = int(math.ceil(n_neurons / 2))
assert group_size == math.floor(n_neurons / 2 + 1)

In [ ]:
combos = list(combinations(prop_weights_indices, group_size))
print(combos[:3])

[(0, 1), (0, 2), (1, 2)]


In [ ]:
cartesian_prod = list(itertools.product(*combos))
print(cartesian_prod)

[(0, 0, 1), (0, 0, 2), (0, 2, 1), (0, 2, 2), (1, 0, 1), (1, 0, 2), (1, 2, 1), (1, 2, 2)]


In [ ]:
clauses_cartesian_products = []
for c in cartesian_prod:
    print(f'Combo: {c}')
    l = [[f'{symb}', f'~{symb}'] for symb in c]
    l_cart_prod = list(itertools.product(*l))
    clauses_cartesian_products += l_cart_prod
    print(l)
    print(l_cart_prod)

Combo: (0, 0, 1)
[['0', '~0'], ['0', '~0'], ['1', '~1']]
[('0', '0', '1'), ('0', '0', '~1'), ('0', '~0', '1'), ('0', '~0', '~1'), ('~0', '0', '1'), ('~0', '0', '~1'), ('~0', '~0', '1'), ('~0', '~0', '~1')]
Combo: (0, 0, 2)
[['0', '~0'], ['0', '~0'], ['2', '~2']]
[('0', '0', '2'), ('0', '0', '~2'), ('0', '~0', '2'), ('0', '~0', '~2'), ('~0', '0', '2'), ('~0', '0', '~2'), ('~0', '~0', '2'), ('~0', '~0', '~2')]
Combo: (0, 2, 1)
[['0', '~0'], ['2', '~2'], ['1', '~1']]
[('0', '2', '1'), ('0', '2', '~1'), ('0', '~2', '1'), ('0', '~2', '~1'), ('~0', '2', '1'), ('~0', '2', '~1'), ('~0', '~2', '1'), ('~0', '~2', '~1')]
Combo: (0, 2, 2)
[['0', '~0'], ['2', '~2'], ['2', '~2']]
[('0', '2', '2'), ('0', '2', '~2'), ('0', '~2', '2'), ('0', '~2', '~2'), ('~0', '2', '2'), ('~0', '2', '~2'), ('~0', '~2', '2'), ('~0', '~2', '~2')]
Combo: (1, 0, 1)
[['1', '~1'], ['0', '~0'], ['1', '~1']]
[('1', '0', '1'), ('1', '0', '~1'), ('1', '~0', '1'), ('1', '~0', '~1'), ('~1', '0', '1'), ('~1', '0', '~1'), ('~1', '~

In [ ]:
clauses_cartesian_products

[('0', '0', '1'),
 ('0', '0', '~1'),
 ('0', '~0', '1'),
 ('0', '~0', '~1'),
 ('~0', '0', '1'),
 ('~0', '0', '~1'),
 ('~0', '~0', '1'),
 ('~0', '~0', '~1'),
 ('0', '0', '2'),
 ('0', '0', '~2'),
 ('0', '~0', '2'),
 ('0', '~0', '~2'),
 ('~0', '0', '2'),
 ('~0', '0', '~2'),
 ('~0', '~0', '2'),
 ('~0', '~0', '~2'),
 ('0', '2', '1'),
 ('0', '2', '~1'),
 ('0', '~2', '1'),
 ('0', '~2', '~1'),
 ('~0', '2', '1'),
 ('~0', '2', '~1'),
 ('~0', '~2', '1'),
 ('~0', '~2', '~1'),
 ('0', '2', '2'),
 ('0', '2', '~2'),
 ('0', '~2', '2'),
 ('0', '~2', '~2'),
 ('~0', '2', '2'),
 ('~0', '2', '~2'),
 ('~0', '~2', '2'),
 ('~0', '~2', '~2'),
 ('1', '0', '1'),
 ('1', '0', '~1'),
 ('1', '~0', '1'),
 ('1', '~0', '~1'),
 ('~1', '0', '1'),
 ('~1', '0', '~1'),
 ('~1', '~0', '1'),
 ('~1', '~0', '~1'),
 ('1', '0', '2'),
 ('1', '0', '~2'),
 ('1', '~0', '2'),
 ('1', '~0', '~2'),
 ('~1', '0', '2'),
 ('~1', '0', '~2'),
 ('~1', '~0', '2'),
 ('~1', '~0', '~2'),
 ('1', '2', '1'),
 ('1', '2', '~1'),
 ('1', '~2', '1'),
 ('1', '

In [ ]:
import numpy as np

print(len(clauses_cartesian_products))
len(np.unique(clauses_cartesian_products))
len(set(clauses_cartesian_products))

64


64

In [ ]:
def encode_prop_features(X):
    n = len(X)
    m = len(X[0])
    prop_features = []
    for i in range(n):
        vals = [i * n + j + len(prop_weights_indices) for j in range(m)]
        prop_features.append(vals)
    

    for i in range(n):
        for j in range(m):
            print(f'X[{i}][{j}] is encoded as {prop_features[i][j]}')

    return prop_features

prop_features = encode_prop_features(X)
print(prop_features)

X[0][0] is encoded as 3
X[0][1] is encoded as 4
X[0][2] is encoded as 5
X[1][0] is encoded as 6
X[1][1] is encoded as 7
X[1][2] is encoded as 8
X[2][0] is encoded as 9
X[2][1] is encoded as 10
X[2][2] is encoded as 11
[[3, 4, 5], [6, 7, 8], [9, 10, 11]]


In [ ]:
unique = [1, 2]
print(get_combinations(unique))

[('w 1', 'x 1', 'w 2', 'x 2'), ('w 1', 'x 1', 'w 2', '~x 2'), ('w 1', 'x 1', '~w 2', 'x 2'), ('w 1', 'x 1', '~w 2', '~x 2'), ('w 1', '~x 1', 'w 2', 'x 2'), ('w 1', '~x 1', 'w 2', '~x 2'), ('w 1', '~x 1', '~w 2', 'x 2'), ('w 1', '~x 1', '~w 2', '~x 2'), ('~w 1', 'x 1', 'w 2', 'x 2'), ('~w 1', 'x 1', 'w 2', '~x 2'), ('~w 1', 'x 1', '~w 2', 'x 2'), ('~w 1', 'x 1', '~w 2', '~x 2'), ('~w 1', '~x 1', 'w 2', 'x 2'), ('~w 1', '~x 1', 'w 2', '~x 2'), ('~w 1', '~x 1', '~w 2', 'x 2'), ('~w 1', '~x 1', '~w 2', '~x 2')]


In [ ]:
clauses_cartesian_products

[('0', '0', '1'),
 ('0', '0', '~1'),
 ('0', '~0', '1'),
 ('0', '~0', '~1'),
 ('~0', '0', '1'),
 ('~0', '0', '~1'),
 ('~0', '~0', '1'),
 ('~0', '~0', '~1'),
 ('0', '0', '2'),
 ('0', '0', '~2'),
 ('0', '~0', '2'),
 ('0', '~0', '~2'),
 ('~0', '0', '2'),
 ('~0', '0', '~2'),
 ('~0', '~0', '2'),
 ('~0', '~0', '~2'),
 ('0', '2', '1'),
 ('0', '2', '~1'),
 ('0', '~2', '1'),
 ('0', '~2', '~1'),
 ('~0', '2', '1'),
 ('~0', '2', '~1'),
 ('~0', '~2', '1'),
 ('~0', '~2', '~1'),
 ('0', '2', '2'),
 ('0', '2', '~2'),
 ('0', '~2', '2'),
 ('0', '~2', '~2'),
 ('~0', '2', '2'),
 ('~0', '2', '~2'),
 ('~0', '~2', '2'),
 ('~0', '~2', '~2'),
 ('1', '0', '1'),
 ('1', '0', '~1'),
 ('1', '~0', '1'),
 ('1', '~0', '~1'),
 ('~1', '0', '1'),
 ('~1', '0', '~1'),
 ('~1', '~0', '1'),
 ('~1', '~0', '~1'),
 ('1', '0', '2'),
 ('1', '0', '~2'),
 ('1', '~0', '2'),
 ('1', '~0', '~2'),
 ('~1', '0', '2'),
 ('~1', '0', '~2'),
 ('~1', '~0', '2'),
 ('~1', '~0', '~2'),
 ('1', '2', '1'),
 ('1', '2', '~1'),
 ('1', '~2', '1'),
 ('1', '

In [ ]:
def get_combinations(unique_combo):
    ws_and_xs = []
    for numb in unique_combo:
        ws_and_xs.append(f'w {numb}')
        ws_and_xs.append(f'x {numb}')
    
    ws_and_xs = [[f'{symb}', f'~{symb}'] for symb in ws_and_xs]
    return list(itertools.product(*ws_and_xs))

def construct_clause_positive(unique_clause_symbols, ):
    clause = []
    # '1' signifies ~w1 V x_i1
    # '~1' signifies w1 V ~x_i1
    for symbol in unique_clause_symbols:  # ('0', '~2', '1')
        if not '~' in symbol:
            # then (e.f. for '1') its not(w_1) V (x_i1)
            index = int(symbol)
            clause.append(-prop_weights_indices[index])
            clause.append(prop_features[i][index])
        elif '~' in symbol:
            # then (e.f. for '1') its (w_1) V not(x_i1)
            symbol = symbol.replace('~', '')
            index = int(symbol)
            clause.append(prop_weights_indices[index])
            clause.append(-prop_features[i][index])
    optimized_clause = list(set(clause))
    return optimized_clause
    
def construct_clause_negative(unique_combo):
    all_combinations = get_combinations(unique_combo)

    clauses = []
    for combination in all_combinations:
        clause = []
        for symbol in combination:
            x_or_w, number = symbol.split(' ')[0], int(symbol.split(' ')[1])
            if x_or_w == 'w':
                clause.append(prop_weights_indices[number])
            if x_or_w == '~w':
                clause.append(-prop_weights_indices[number])
            if x_or_w == 'x':
                clause.append(prop_features[i][number])
            if x_or_w == '~x':
                clause.append(-prop_features[i][number])
        clauses.append(clause)
    return clauses


wcnf = WCNF()
for i in range(len(X)):
    if Y[i] is True:
        for c in clauses_cartesian_products:
            # ('0', '0', '~1')
            # Here we optimize - we dont need to add identical clauses
            unique_clause_symbols = set(c)  # ('0', '0', '1') --> ('0', '1')
            
            # print(set(c), c)
            clause = construct_clause_positive(unique_clause_symbols)
            wcnf.append(clause, weight = 1)
            # print(f'Our clause: {clause}')
    
    elif Y[i] is False:
        # print(f'Y is false')
        for c in cartesian_prod: # (1, 2) (2, 3) (1, 3) --> (1, 2, 1), (1, 2, 3), (1, 3, 1), ...
            # print(f'combo: {c}')
            unique_combo = set(c)  # (1, 2) --> [(~w1 & x_i1) V (w1 & ~x_i1)] V [(~w2 & x_i2) V (w2 & ~x_i2)]
            # print(f'unique combo: {set(unique_combo)}')

            clauses = construct_clause_negative(unique_combo)
            for c in clauses:
                wcnf.append(c, weight = 1)
    # print(clause)
    

In [ ]:
get_combinations([0, 1])

[('w 0', 'x 0', 'w 1', 'x 1'),
 ('w 0', 'x 0', 'w 1', '~x 1'),
 ('w 0', 'x 0', '~w 1', 'x 1'),
 ('w 0', 'x 0', '~w 1', '~x 1'),
 ('w 0', '~x 0', 'w 1', 'x 1'),
 ('w 0', '~x 0', 'w 1', '~x 1'),
 ('w 0', '~x 0', '~w 1', 'x 1'),
 ('w 0', '~x 0', '~w 1', '~x 1'),
 ('~w 0', 'x 0', 'w 1', 'x 1'),
 ('~w 0', 'x 0', 'w 1', '~x 1'),
 ('~w 0', 'x 0', '~w 1', 'x 1'),
 ('~w 0', 'x 0', '~w 1', '~x 1'),
 ('~w 0', '~x 0', 'w 1', 'x 1'),
 ('~w 0', '~x 0', 'w 1', '~x 1'),
 ('~w 0', '~x 0', '~w 1', 'x 1'),
 ('~w 0', '~x 0', '~w 1', '~x 1')]

In [ ]:
construct_clause_negative([0, 1])

[[0, 3, 1, 4],
 [0, 3, 1, -4],
 [0, 3, -1, 4],
 [0, 3, -1, -4],
 [0, -3, 1, 4],
 [0, -3, 1, -4],
 [0, -3, -1, 4],
 [0, -3, -1, -4],
 [0, 3, 1, 4],
 [0, 3, 1, -4],
 [0, 3, -1, 4],
 [0, 3, -1, -4],
 [0, -3, 1, 4],
 [0, -3, 1, -4],
 [0, -3, -1, 4],
 [0, -3, -1, -4]]

In [ ]:
cartesian_prod

[(0, 0, 1),
 (0, 0, 2),
 (0, 2, 1),
 (0, 2, 2),
 (1, 0, 1),
 (1, 0, 2),
 (1, 2, 1),
 (1, 2, 2)]

In [ ]:
len(wcnf.soft)

352

In [ ]:
# Weighted CNF form
wcnf = WCNF()


for x, y in zip(X, Y):
    print(f'Encoding x = {x}, y = {y}')
    
    # At that step, we have combinations of weight indices
    # For example, for 3 weights
    # [(w0 == x_i0) & (w1 == x_i1)] V [(w0 == x_i0) & (w2 == x_i2)] V [(w1 == x_i1) & (w2 == x_i2)]

    # Each combination in cartesian_prod(w_combinations) - is a special clause:
    # For example, for n_weights == 3, after cartesian product, all combinations:
    # (001) (002) (021) (022) (101) (102) (121) (122)
    # Which means (for the first clause (001)):
    # (w_0 == x_i0) | (w_0 == x_i0) | (w_1 == x_i1)

    # However, we still did not expand the equivalence (==):
    # [(~w_0 V x_i0) & (w_0 V ~x_i0)] V [(~w_0 V x_i0) & (w_0 V ~x_i0)] V [(~w_1 V x_i1) & (w_1 V ~x_i1)]

    # And this also has to be transformed into CNF:
    # Do substitution:
    # [(0) & (~0)] V [(0) & (~0)] V [(1) & (~1)]
    # Where 0: ~w_0 V x_i0, ~0: w_0 V ~x_i0
    # We have to transfer (001) -> [[0, ~0], [0, ~0], [1, ~1]] and make a cartesian product of it

    

    

    # for w_combination in combinations(propositional_weights, group_size):
    #     # print(c, c[0], c[1])
    #     # print(propositional_weights.index(c[0]), propositional_weights.index(c[1]))
    #     print(f'weight combination = {w_combination}')
        
    #     for weight in w_combination:
    #         # index of weight --> the same as index of feature of current object x_i in the X dataset
    #         w_index = propositional_weights.index(weight)
    #         x_feature = x[w_index]
    #         print(f'w_{w_index}, x_j = {x_feature}')

    #         # Our clause is - weight == x_feature (equivalence)
    #         # which is (not(weight) V x_feature) and (not(x_feature) V weight)
            
    #         first_clause = [-weight, x_feature]
    #         second_clause = [weight, -x_feature]
    #         wcnf.append



NameError: ignored

In [ ]:
# No hard clauses are introduced.
# Soft clauses - have weight 1, if the prediction of the network is correct
# for a given instance (x_i, y_i) in a dataset

In [ ]:
# Encode this into clauses
for x_i, y_i in zip(X, Y):
    print(f'Encoding x_i = {x_i}, y_i = {y_i}')
    

Encoding x_i = [True, False, False], y_i = False
Encoding x_i = [False, True, False], y_i = True
Encoding x_i = [True, True, False], y_i = True
